In [1]:
import os
import numpy as np
from scipy.io import loadmat, savemat
from tqdm.auto import tqdm

from utils.preprocess_tools import calculate_csi_ratio, hampel_filter, phase_calibration

d:\Anaconda\envs\Deep_Learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '../CSI-Data-Complex/env3'
save_path = '../CSI-Ratio/env3'

In [3]:
users = os.listdir(data_path)

In [4]:
for user in users:
    data_user_path = os.path.join(data_path, user)
    save_user_path = os.path.join(save_path, user)
    if not os.path.exists(save_user_path):
        os.makedirs(save_user_path)
    
    for file in tqdm(os.listdir(data_user_path)):
        if os.path.exists(os.path.join(save_user_path, file)):
            continue
        try:
            origin_csi = loadmat(os.path.join(data_user_path, file))['csi_data']
            origin_csi = origin_csi.reshape(origin_csi.shape[0], 3, -1)
            csi_ratio, antenna_index = calculate_csi_ratio(origin_csi)
            csi_ratio = np.concatenate((csi_ratio[:, :antenna_index, :], csi_ratio[:, antenna_index+1:, :]), axis=1)    
            angle_csi_ratio = np.angle(csi_ratio)
            angle_csi_ratio = phase_calibration(hampel_filter(angle_csi_ratio))
            angle_csi_ratio = angle_csi_ratio.reshape(angle_csi_ratio.shape[0], -1)
            savemat(os.path.join(save_user_path, file), {'csi-ratio':angle_csi_ratio})
        except:
            continue        



100%|██████████| 750/750 [18:26<00:00,  1.47s/it]  
